In [52]:
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta
import random

random.seed(123)

# Data load
df = pd.read_csv('/superstore.csv', encoding='latin1')
print(df.head()) # Data check
print(df.columns) # Columns check
      
# Ensure required columns exist 
required_cols = ['Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']
for c in required_cols: 
    if c not in df.columns: 
        raise SystemExit(f"Missing column: {c}")

# Rename columns to safe names 
df = df[required_cols].rename(columns={ 'Order ID':'order_id','Order Date':'order_date','Ship Date':'ship_date',
                                       'Ship Mode':'ship_mode', 'Customer ID':'customer_id','Customer Name':'customer_name',
                                       'Segment':'segment','Country':'country', 'City':'city','State':'state','Postal Code':'postal_code', 
                                       'Region':'region','Product ID':'product_id','Category':'category', 'Sub-Category':'sub_category',
                                       'Product Name':'product_name','Sales':'sales','Quantity':'quantity', 'Discount':'discount','Profit':'profit' })
print(df.head()) # Column renaming check

# Convert dates 
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce') 
df['ship_date'] = pd.to_datetime(df['ship_date'], errors='coerce')

# Create initial load (a random snapshot)
initial = df.copy().reset_index(drop=True)
# Save initial load 
initial.to_csv('/initial_load.csv', index=False)

   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
2       3  CA-2016-138688   6/12/2016   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0       42420   Sout

In [39]:
# --- Create secondary load with: 
# - some new records 
# - some SCD1 changes (customer name corrected) 
# - some SCD2 changes (customer region changed) 
# - some exact duplicates of initial rows 
# - some modified orders (e.g., changed profit value) for SCD1-like behavior


secondary = pd.DataFrame(columns=initial.columns)
print(secondary)

n = len(initial)
print(n)

Empty DataFrame
Columns: [order_id, order_date, ship_date, ship_mode, customer_id, customer_name, segment, country, city, state, postal_code, region, product_id, category, sub_category, product_name, sales, quantity, discount, profit]
Index: []
9994


In [40]:
# Keep 70% of rows unchanged (simulate unchanged shipments) 
unchanged_idx = initial.sample(frac=0.70, random_state=1).index 
secondary = pd.concat([secondary, initial.loc[unchanged_idx]], ignore_index=True)

print(secondary)

            order_id order_date  ship_date       ship_mode customer_id  \
0     CA-2015-131597 2015-09-14 2015-09-18  Standard Class    SP-20620   
1     CA-2015-129098 2015-10-09 2015-10-13  Standard Class    DK-13090   
2     US-2014-147627 2014-01-20 2014-01-26  Standard Class    HL-15040   
3     CA-2015-128125 2015-03-31 2015-04-05  Standard Class    EB-13705   
4     CA-2014-113929 2014-06-16 2014-06-21  Standard Class    CK-12205   
...              ...        ...        ...             ...         ...   
6991  CA-2014-149958 2014-03-15 2014-03-19  Standard Class    AS-10240   
6992  CA-2014-127012 2014-08-11 2014-08-15  Standard Class    GM-14680   
6993  CA-2017-169404 2017-04-09 2017-04-14  Standard Class    NC-18625   
6994  CA-2017-142636 2017-11-03 2017-11-07  Standard Class    KC-16675   
6995  CA-2015-115742 2015-04-18 2015-04-22  Standard Class    DP-13000   

            customer_name    segment        country         city  \
0        Stefania Perrino  Corporate  Unite

/var/folders/1x/kg3kyc1n25vcq0b26636jq040000gn/T/ipykernel_2331/2772392141.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  secondary = pd.concat([secondary, initial.loc[unchanged_idx]], ignore_index=True)


In [41]:
# New records: 3% new orders with new order_id and new customer_id 
new_count = int(n * 0.03) # 3% of the dataset will be new rows
sample_for_template = initial.sample(n=new_count, random_state=5) # Pick some rows from the initial dataset as templates
new_rows = sample_for_template.copy() 
for i in range(len(new_rows)): # Create new order id and new customer id 
    new_rows.iloc[i, new_rows.columns.get_loc('order_id')] = f"NEWORD{100000 + i}" # Assign a new unique order ID like 'NEWWORD01'
    new_rows.iloc[i, new_rows.columns.get_loc('customer_id')] = f"NEWCUST{10000 + i}" # Assign a new unique customer ID like 'NEWCUST01'
    new_rows.iloc[i, new_rows.columns.get_loc('customer_name')] = f"New Customer {i}" # Assign a name like 'New Customer 0' etc.
    # shift date forward 
    new_rows.iloc[i, new_rows.columns.get_loc('order_date')] = new_rows.iloc[i]['order_date'] + pd.Timedelta(days=30) # Add 30 days to the original order_date to look like new orders
secondary=pd.concat([secondary,new_rows], ignore_index=True)
print(secondary)

            order_id order_date  ship_date       ship_mode   customer_id  \
0     CA-2015-131597 2015-09-14 2015-09-18  Standard Class      SP-20620   
1     CA-2015-129098 2015-10-09 2015-10-13  Standard Class      DK-13090   
2     US-2014-147627 2014-01-20 2014-01-26  Standard Class      HL-15040   
3     CA-2015-128125 2015-03-31 2015-04-05  Standard Class      EB-13705   
4     CA-2014-113929 2014-06-16 2014-06-21  Standard Class      CK-12205   
...              ...        ...        ...             ...           ...   
7290    NEWORD100294 2017-04-26 2017-03-29    Second Class  NEWCUST10294   
7291    NEWORD100295 2017-12-09 2017-11-12    Second Class  NEWCUST10295   
7292    NEWORD100296 2014-06-03 2014-05-05     First Class  NEWCUST10296   
7293    NEWORD100297 2017-01-02 2016-12-07  Standard Class  NEWCUST10297   
7294    NEWORD100298 2016-10-17 2016-09-22  Standard Class  NEWCUST10298   

            customer_name    segment        country         city       state  \
0      

In [42]:
# Create SCD1 changes: customer_name corrections for some customer_ids (3%) 
s1_idx = initial.sample(frac=0.03, random_state=3).index # Select some rows to modify
s1_rows = initial.loc[s1_idx].copy() 
# Flip a character in customer_name to simulate correction  
s1_rows['customer_name'] = s1_rows['customer_name'].apply(lambda x: f"Corrected {x}") 
secondary = pd.concat([secondary,s1_rows], ignore_index=True)
print(secondary)

            order_id order_date  ship_date       ship_mode customer_id  \
0     CA-2015-131597 2015-09-14 2015-09-18  Standard Class    SP-20620   
1     CA-2015-129098 2015-10-09 2015-10-13  Standard Class    DK-13090   
2     US-2014-147627 2014-01-20 2014-01-26  Standard Class    HL-15040   
3     CA-2015-128125 2015-03-31 2015-04-05  Standard Class    EB-13705   
4     CA-2014-113929 2014-06-16 2014-06-21  Standard Class    CK-12205   
...              ...        ...        ...             ...         ...   
7590  CA-2016-133669 2016-10-21 2016-10-27  Standard Class    SV-20935   
7591  CA-2016-125017 2016-02-06 2016-02-11  Standard Class    KB-16240   
7592  CA-2016-125843 2016-08-13 2016-08-16     First Class    RF-19840   
7593  CA-2014-146283 2014-09-08 2014-09-15  Standard Class    KT-16465   
7594  CA-2014-145317 2014-03-18 2014-03-23  Standard Class    SM-20320   

                  customer_name      segment        country          city  \
0              Stefania Perrino   

In [43]:
# Create SCD2 changes: same customer_id but different region for some rows (2%) 
s2_idx = initial.sample(frac=0.02, random_state=3).index 
s2_rows = initial.loc[s2_idx].copy() 
# Change region to a different valid value (choose one not equal to original) 
all_regions = initial['region'].unique().tolist() 
def different_region(r): 
    choices = [x for x in all_regions if x != r] 
    return random.choice(choices) if choices else r 
s2_rows['region'] = s2_rows['region'].apply(different_region) 
# Update order_date to a later date in order to simulate later event 
s2_rows['order_date'] = s2_rows['order_date'] + pd.Timedelta(days=180) 
secondary = pd.concat([secondary,s2_rows], ignore_index=True)
print(secondary)

            order_id order_date  ship_date       ship_mode customer_id  \
0     CA-2015-131597 2015-09-14 2015-09-18  Standard Class    SP-20620   
1     CA-2015-129098 2015-10-09 2015-10-13  Standard Class    DK-13090   
2     US-2014-147627 2014-01-20 2014-01-26  Standard Class    HL-15040   
3     CA-2015-128125 2015-03-31 2015-04-05  Standard Class    EB-13705   
4     CA-2014-113929 2014-06-16 2014-06-21  Standard Class    CK-12205   
...              ...        ...        ...             ...         ...   
7790  CA-2016-109666 2016-10-16 2016-04-26  Standard Class    KM-16720   
7791  CA-2015-126557 2016-01-08 2015-07-17    Second Class    RL-19615   
7792  CA-2014-159835 2015-05-16 2014-11-24  Standard Class    RB-19330   
7793  CA-2014-131947 2015-03-16 2014-09-22  Standard Class    JA-15970   
7794  CA-2015-149909 2016-05-11 2015-11-17  Standard Class    RA-19915   

            customer_name    segment        country           city  \
0        Stefania Perrino  Corporate  Uni

In [45]:
# Create exact duplicates (5% of initial)
dup_idx = initial.sample(frac=0.05, random_state=2).index
secondary = pd.concat([secondary, initial.loc[dup_idx]], ignore_index=True)
print(secondary)

            order_id order_date  ship_date       ship_mode customer_id  \
0     CA-2015-131597 2015-09-14 2015-09-18  Standard Class    SP-20620   
1     CA-2015-129098 2015-10-09 2015-10-13  Standard Class    DK-13090   
2     US-2014-147627 2014-01-20 2014-01-26  Standard Class    HL-15040   
3     CA-2015-128125 2015-03-31 2015-04-05  Standard Class    EB-13705   
4     CA-2014-113929 2014-06-16 2014-06-21  Standard Class    CK-12205   
...              ...        ...        ...             ...         ...   
8790  CA-2017-144862 2017-12-01 2017-12-05  Standard Class    EH-14005   
8791  CA-2015-137064 2015-02-06 2015-02-13  Standard Class    TS-21655   
8792  CA-2017-114370 2017-03-14 2017-03-17    Second Class    BN-11470   
8793  CA-2014-127166 2014-05-21 2014-05-23    Second Class    KH-16360   
8794  CA-2016-103730 2016-06-12 2016-06-15     First Class    SC-20725   

            customer_name      segment        country          city  \
0        Stefania Perrino    Corporate  

In [51]:
# Shuffle the secondary and save as a csv file
secondary = secondary.sample(frac=1, random_state=7).reset_index(drop=True) 
secondary.to_csv('/secondary_load.csv', index=False)

In [47]:
print("Created initial_load.csv (rows: {})".format(len(initial))) 
print("Created secondary_load.csv (rows: {})".format(len(secondary)))

Created initial_load.csv (rows: 9994)
Created secondary_load.csv (rows: 8795)
